In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt


In [2]:
cols = ['Cycle', 'Date', 'Time', 'RelTime[s]'] + [str(i+1) for i in range(20)]

In [3]:
data = pd.read_csv('/content/wycr 3.csv', sep = '\t', names = cols)

FileNotFoundError: [Errno 2] No such file or directory: '/content/wycr 3.csv'

In [ ]:
data = data.iloc[1:]
data = data.drop(['20'], axis = 1)
cols_to_float = ['RelTime[s]'] + [str(i+1) for i in range(19)]
data[cols_to_float] = data[cols_to_float].astype(float)

In [ ]:
import datetime

In [ ]:
datetime_start = datetime.datetime.strptime('16:23:37', '%H:%M:%S')


In [ ]:
data = data[data['Time'].apply(lambda x: datetime.datetime.strptime(x, '%H:%M:%S:%f'))>datetime_start]

In [ ]:
#Date and time: 17.02.2022 16:23:37


In [ ]:
temp = pd.read_csv('/content/temp_WYCr 3.txt', sep="\t")
for col in temp.columns:
  temp[col] = temp[col].apply(lambda x: float(x.replace(',','.')))

In [ ]:
data['RelTime[s]'] = data['RelTime[s]'] - data['RelTime[s]'].iloc[0]

In [ ]:
data.head()

In [ ]:
temp.head()

In [ ]:
titles = [3, 4, 19, 20]
arr_of_integrals = []
for idx, need_col in enumerate(['2', '3', '12', '13']):
  x = data['RelTime[s]']
  y = data[need_col]
  arr_of_integrals.append(np.trapz(y, x))
  x1 = temp['t, s']
  y1 = temp['Tr, C'] + 273
  fig,ax = plt.subplots()
  ax.plot(x, y, color='teal',linewidth=3, label = "I(t)")
  ax.grid()


  ax.set_title(f'График зависимости тока и температуры от времени\n для массы {titles[idx]}')
  ax.set_ylabel('I, A')
  ax.set_xlabel('t, С')
  ax2 = ax.twinx()
  ax2.plot(x1, y1, color='orange', linewidth=3, label = "T(t)")

  # Align to 0
  #ax2.set_ylim(bottom=0)
  ax2.set_ylabel('T, K')
  ax.legend(loc='center right')
  ax2.legend(loc='upper left')


  plt.show()


In [ ]:
arr_of_integrals

In [ ]:
calibr = pd.read_csv('/content/calibr_(wycr 3).csv', sep = '\t', names = cols)

In [ ]:
calibr = calibr.iloc[2:]
calibr = calibr.drop(['20'], axis = 1)
cols_to_float = ['RelTime[s]'] + [str(i+1) for i in range(19)]
calibr[cols_to_float] = calibr[cols_to_float].astype(float)
calibr = calibr[calibr['RelTime[s]'] > temp['t, s'].iloc[0]]

In [ ]:
y_calibr = calibr['3']
x_calibr = calibr['RelTime[s]']

In [ ]:
I0 = calibr[ calibr['RelTime[s]'] > 800]['3'].mean()
I0 = [I0 for i in range(len(x_calibr))]

In [ ]:
plt.plot(x_calibr, y_calibr, label='I(t)')
plt.plot(x_calibr, I0,  '--', color='red', linewidth=2 , label='I0')
plt.xlabel('t, c')
plt.ylabel('I, A')
plt.title('Калибровочный график для массы 4')
plt.ylim([0,0.4e-7])
plt.grid()

plt.legend()
plt.show()

In [ ]:
baratron = pd.read_csv('/content/baratron_(wycr 3).txt', sep="\t", names = ['t','P'])
baratron['t'] = baratron['t'].apply(lambda x: float(x.replace(',','.')))
baratron['P'] = baratron['P'].apply(lambda x: float(x.replace(',','.')))

In [ ]:
x_bara = baratron['t']
y_bara = baratron['P'] * 0.750062 # to Torr

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def objective(x, a, b):
	return a * x + b

In [ ]:
fit_x = x_bara[150:400]
fit_y = y_bara[150:400]

In [ ]:
popt, _ = curve_fit(objective, fit_x, fit_y)
a, b = popt

In [ ]:
x_line = np.arange(min(fit_x), max(fit_x), 1)

In [ ]:
y_line = objective(x_line, a, b)

In [ ]:
plt.plot(x_line, y_line, '--', color='red', linewidth=3, label = "fitting")
plt.plot(x_bara, y_bara, label = "P(t)")
plt.xlabel('t, c')
plt.grid()
plt.ylabel('P, Torr')
plt.title('График зависимости давления от времени')
plt.legend()
plt.show()

In [ ]:
a, I0[0]

In [ ]:
 5.2e-2 * a/I0[0]

In [ ]:
k = 3.5e19 * 5.2e-2 * a/I0[0]
k

In [ ]:
integrals_prod_coef = k * np.array(arr_of_integrals)

In [ ]:
coef = [0.5, 2, 0.5, 2]
for idx, c in enumerate(coef):
  integrals_prod_coef[idx]*=c

In [ ]:
integrals_prod_coef

In [ ]:
integrals_prod_coef.sum()/1e15